In [ ]:
! pip install datasets transformers tiktoken pinecone-client langchain openai

In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration
from datasets import load_dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large").to(device)

In [ ]:
datasets = load_dataset("openlifescienceai/medmcqa")

In [ ]:
datasets

In [ ]:
def preprocess_and_tokenize(examples):
  input_sequences = []
  references = []

  for example in examples:
    question = "Instructions: The following is a multiple choice question about medical knowledge. Solve it in a step-by-step fashion, starting by summarizing the available information. Output a single option from the four options as the final answer. Question: " + example["question"] + " (A) " + example["opa"] + " (B) " + example["opb"] + " (C) " + example["opc"] + " (D) " + example["opd"]
    answer = [example["opa"], example["opb"], example["opc"], example["opd"]][example["cop"]]

    input_sequences.append(question)
    references.append(answer)

  input_sequences_and_references = [(sequence, reference) for sequence, reference in zip(input_sequences, references) if is_shorter_than_max_length(sequence)]
  input_sequences, references = zip(*input_sequences_and_references)

  tokenized_input_sequences = tokenizer(
      input_sequences,
      max_length=512,
      padding=True,
      return_tensors="pt"
  )

  input_ids, attention_masks = tokenized_input_sequences.input_ids, tokenized_input_sequences.attention_mask

  labels = tokenizer(references, padding="longest", return_tensors="pt").input_ids
  labels[labels == tokenizer.pad_token_id] = -100

  return input_ids, attention_masks, labels, references

def is_shorter_than_max_length(sequence):
  inputs = tokenizer(sequence, truncation=False)
  return len(inputs.input_ids) <= 128

# Training

In [ ]:
from datasets import Dataset
input_ids, attention_masks, labels, _ = preprocess_and_tokenize(datasets["train"])

val_input_ids, val_attention_masks, val_labels, _ = preprocess_and_tokenize(datasets["validation"])

Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

max_source_length = 512
max_target_length = 128
batch_size = 2
num_epochs = 1

num_training_steps_per_epoch = len(input_ids) // batch_size
num_training_steps = num_training_steps_per_epoch * num_epochs

progress_bar = tqdm(range(num_training_steps))

optimizer = AdamW(model.parameters(), lr=3e-5)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

val_losses = []

for i in range(num_epochs):
  val_losses.append(model(input_ids=val_input_ids[i*8:i*8+8].to(device), attention_mask=val_attention_masks[i*8:i*8+8].to(device), labels=val_labels[i*8:i*8+8].to(device)).loss)

  for j in range(0, num_training_steps_per_epoch * batch_size, batch_size):
    batch_input_ids = input_ids[j:j+batch_size].to(device)
    batch_attention_masks = attention_masks[j:j+batch_size].to(device)
    batch_labels = labels[j:j+batch_size].to(device)

    loss = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels).loss

    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

val_losses.append(model(input_ids=val_input_ids[3*8:3*8+8].to(device), attention_mask=val_attention_masks[3*8:3*8+8].to(device), labels=val_labels[3*8:3*8+8].to(device)).loss)

In [ ]:
for l in val_losses:
  print(l.item())

# Inference

In [ ]:
evaluation_input_ids, _, _, references = preprocess_and_tokenize(datasets["validation"])
prediction_ids = []
for i in range(1000):
  prediction_ids.append(model.generate(evaluation_input_ids[i:i + 1].to(device))[0])
predictions = [tokenizer.decode(prediction_id, skip_special_tokens=True) for prediction_id in prediction_ids]
predictions = ["" if len(prediction)== 0 else prediction.lower() if prediction[-1] != '.' else prediction[:-1].lower() for prediction in predictions]
references = [reference.lower() if reference[-1] != '.' else reference[:-1].lower() for reference in references[:1000]]

In [ ]:
prompt = "Which of the following is not true for myelinated nerve fibers: (A) Impulse through myelinated fibers is slower than non-myelinated fibers (B) Membrane currents are generated at nodes of Ranvier (C) Saltatory conduction of impulses is seen (D) Local anesthesia is effective only when the nerve is not covered by myelin sheath?"
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
outputs = model.generate(**inputs)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

# Evaluation

In [ ]:
# fixes some wierd bug for below pip install evaluate
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%pip install evaluate
%pip install git+https://github.com/google-research/bleurt.git
%cd bleurt
%pip install .

In [ ]:
import evaluate
import numpy as np

bleu = evaluate.load("bleu")
bleurt = evaluate.load("bleurt", module_type="metric")

bleu_references = [[reference] for reference in references.copy()]

bleu1_results = bleu.compute(predictions=predictions, references=bleu_references, max_order=1)
bleu2_results = bleu.compute(predictions=predictions, references=bleu_references, max_order=2)
bleu3_results = bleu.compute(predictions=predictions, references=bleu_references, max_order=3)
bleu4_results = bleu.compute(predictions=predictions, references=bleu_references, max_order=4)

bleurt_results = bleurt.compute(predictions=predictions, references=references)

print(bleu1_results)
print(bleu2_results)
print(bleu3_results)
print(bleu4_results)
print(np.mean(bleurt_results["scores"]))

for i in range(10):
  print(predictions[i], "|", references[i], "|", bleurt.compute(predictions=predictions[i:i+1], references=references[i:i+1])["scores"][0])

In [ ]:
cnt = 0
for i in range(100):
  if predictions[i] == "(a)" or predictions[i] == "(b)" or predictions[i] == "(c)" or predictions[i] == "(d)":
    cnt += 1
print(cnt)

In [ ]:
for i in range(100):
  print(predictions[i], "|", references[i])

In [ ]:
from evaluate import load
exact_match_metric = load("exact_match")
results = exact_match_metric.compute(predictions=predictions, references=references)
print(results)

In [ ]:
def calculate_precision(prediction_tokens, reference_tokens):
    prediction_set = set(prediction_tokens)
    reference_set = set(reference_tokens)

    # Calculate the number of common elements (intersection)
    common_tokens = prediction_set.intersection(reference_set)
    num_common = len(common_tokens)

    # Calculate precision and recall
    if len(prediction_set) == 0:
        precision = 0
    else:
        precision = num_common / len(prediction_set)

    return precision

def calculate_recall(prediction_tokens, reference_tokens):
    prediction_set = set(prediction_tokens)
    reference_set = set(reference_tokens)

    # Calculate the number of common elements (intersection)
    common_tokens = prediction_set.intersection(reference_set)
    num_common = len(common_tokens)

    if len(reference_set) == 0:
        recall = 0
    else:
        recall = num_common / len(reference_set)

    return recall

def calculate_f1_score(prediction_tokens, reference_tokens):
    precision = calculate_precision(prediction_tokens, reference_tokens)
    recall = calculate_recall(prediction_tokens, reference_tokens)

    if (precision + recall) == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    return f1_score

In [ ]:
precisions = []
recalls = []
f1s = []
for i in range(100):
  precisions.append(calculate_precision(tokenizer(predictions[i])["input_ids"], tokenizer(references[i])["input_ids"]))
  recalls.append(calculate_recall(tokenizer(predictions[i])["input_ids"], tokenizer(references[i])["input_ids"]))
  f1s.append(calculate_f1_score(tokenizer(predictions[i])["input_ids"], tokenizer(references[i])["input_ids"]))
print(np.mean(precisions))
print(np.mean(recalls))
print(np.mean(f1s))

In [ ]:
pip install rouge_score

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions,references=references)
print(results)